# WEEK 4 Stacks and Queues

This notebook demonstrates uses of stacks and queues, the simple but powerful data structures used for managing data in a Last In/First Out (LIFO) or First In/First Out (FIFO) manner.

## Section 1: Postfix Notation Introduction

As citizens of an arithmetic savvy world, we're used to seeing our mathematical expressions in _infix_ notation, where the Operator sits in between the two Operands, resulting in expressions that look like this: $A + B$ or $C * D$.  To execute expressions in infix notation, you must have an understanding of Order of Operations.  For example, $4 * (3 + 2)$ you know that the addition in the parenthesis is performed first.  This is good for people, but not as simple for computers.

There is an alternative, though, called _postfix_ notation, where the Operator comes **after** the Operands.  Instead of $A + B$, we write $A B +$.  Using _postfix_ notation, we can use a stack to evaluate the expression.

Another, less trivial example is $15 + 3 * 4 - 5 = 22$.  This expression would be written $15$ $3$ $4$ $*$ $+$  $5$ $-$ in postfix notation.

The idea is that you will push values onto a stack until you see an operator, in which case you will perform said operation on the previous two numbers.

### Create a Stack Data Structure
To get started, we are going to create a basic stack data structure.

In [28]:
class Stack:
    class StackNode:
        def __init__(self, data=None):
            self.data = data
            self.next = None
    
    def __init__(self):
        self.top = None
        self.size = 0
            
    def is_empty(self):
        return self.size == 0

    def push(self, data):
        n = Stack.StackNode(data)
        
        if self.is_empty():
            self.top = n
        else:
            n.next = self.top
            self.top = n
            
        self.size += 1
        
    def pop(self):
        if self.is_empty():
            return None
        else:
            r = self.top
            self.top = self.top.next
            self.size -= 1
            return r.data
        
    def peek(self):
        return self.top.data if not self.is_empty() else None
        

With our basic stack data structure, we can write an expression taht will perform the supported operations, $+, -, *, /, ^$

For the sake of simplicity we are goign to assume the expression has already been validated as a properly formed expression, consisting of only operators (numbers) and operands ($+, -, *, /, ^$) separated by spaces.

For example: ```15 3 4 * + 5 -```

In [29]:
def do_math(a, b, op):
    af = float(a)
    bf = float(b)
    
    if op == "+":
        return af+bf
    elif op == "-":
        return af-bf
    elif op == "*":
        return af*bf
    elif op == "/":
        return af/bf
    elif op == "^":
        return af**bf


def solve_postfix(expr):
    s = Stack()
    
    # SPLITTING BY SPACES ALLOWS US TO DO WORK ON VALUES > 10, AND DO FLOATING POINT MATH
    expr_split = expr.split(" ")
    
    operators = ["+", "-", "*", "/", "^"]
    
    for ch in expr_split:
        
        if ch in operators:
            b = s.pop()
            a = s.pop()
            s.push(do_math(a,b,ch))
        else:
            s.push(ch)
            
    return s.pop()

### Testing out the Expression

Recall the infix example $15 + 3 * 4 - 5 = 22$.  Which in postfix notation is written:$15$ $3$ $4$ $*$ $+$  $5$ $-$.

Let's feed that into the ```postfix()``` function and see if we get the correct answer.


In [30]:
expr = "15 3 4 * + 5 -"

expected_answer = 22

result = solve_postfix(expr)

print(f"We expected: {expected_answer}, Got: {result}, Match T/F: {expected_answer == result}")

We expected: 22, Got: 22.0, Match T/F: True


### Looks Great!

The test came out as expected.  Now for the more complicated part.  Converting an infix expression into a postfix expression so that is can be handled by a stack.

## Section 2:  Creating an Infix to Postfix Converter

To create a infix converter, we need to setup a few things. First we need to know the order of operations as mentioned before.  So we set this up with setting priority:  $+$ and $-$ are lowest priority, $*$ and $/$ are middle priority and $^$ is the highest.  We will also handle parenthesis so that the inner most expressions are handled first.

We will also assume that the input is a set of operators and operands, separated by spaces, the same as our postfix calculator.

### Using a Stack
We will use a stack to keep track of the parenthesis as we come across them, and to make sure the operators are placed in the proper order.

In [31]:
def infix_to_postfix(expr):
    operators = ["+", "-", "*", "/", "^", "(", ")"]

    order_of_ops = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}

    s = Stack()

    expr_split = expr.split(" ")

    postfix_expr = ''

    for ch in expr_split:
        if ch not in operators:
            # place the operand on the postfix_expression
            postfix_expr += ch + " "
        elif ch == "(":
            # new open paren, push it onto the stack
            s.push(ch)
        elif ch == ")":
            # closing paren, pop items until you see a matching paren:
            while not s.is_empty() and s.peek() != '(':
                postfix_expr += s.pop() + " "
            s.pop()  # drop the matching "parenthesis on the floor"
        else:
            # we have an operator, so keep prop them in priority order.
            while not s.is_empty() and s.peek() != '(' and \
                    order_of_ops[ch] <= order_of_ops[s.peek()]:
                postfix_expr += s.pop() + " "

            # push the character back
            s.push(ch)

    while not s.is_empty():
        # if there are remaining operators still on the stack, add them to the end.
        postfix_expr += s.pop() + " "

        
    return postfix_expr[:-1]  # return all but the final char which is a space.

We can now use the examples from the beginning to see if we are converting things correctly.

$4 + (3 + 2)$ in Postfix Notation would be: $4$ $3$ $2$ $+$ $*$

$15 + 3 * 4 - 5 = 22$ in Postfix Notation would be $15$ $3$ $4$ $*$ $+$  $5$ $-$.

Let's try it out:

In [32]:
infix_expr = '4 * ( 3 + 2 )'
postfix_expr = infix_to_postfix(infix_expr)
result = solve_postfix(postfix_expr)

print("Easy Example")
print(f"Infix: {infix_expr}\nPostfix: {postfix_expr}\nResult: {result}")

infix_expr = '15 + 3 * 4 - 5'
postfix_expr = infix_to_postfix(infix_expr)
result = solve_postfix(postfix_expr)

print("Harder Example")
print(f"Infix: {infix_expr}\nPostfix: {postfix_expr}\nResult: {result}")


Easy Example
Infix: 4 * ( 3 + 2 )
Postfix: 4 3 2 + *
Result: 20.0
Harder Example
Infix: 15 + 3 * 4 - 5
Postfix: 15 3 4 * + 5 -
Result: 22.0


Let's try a harder one that includes nested paranthesis and exponents.

In [27]:
infix_expr = '( 5 + 4 ) * ( 5 + 4 ^ 2 * ( 2 + 2 ) - 100 ) + 279'
postfix_expr = infix_to_postfix(infix_expr)

result = solve_postfix(postfix_expr)
print(f"Infix: {infix_expr}\nPostfix: {postfix_expr}\nResult: {result}")


Infix: ( 5 + 4 ) * ( 5 + 4 ^ 2 * ( 2 + 2 ) - 100 ) + 279
Postfix: 5 4 + 5 4 2 ^ 2 2 + * + 100 - * 279 +
Result: 0.0
